# 1.- Importar todas la librerias

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os
import time

# 2.- Iniciar Sesion en IG

In [2]:
# Cargar variables desde .env
load_dotenv()

chrome_options = Options()
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://www.instagram.com/")

# Add account credentials
user_ = os.getenv('IG_USERNAME')
pass_ = os.getenv('IG_PASSWORD')

# Locate username and password input fields
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# Input username y password
username.send_keys(user_)
password.send_keys(pass_)

# Locate 'log in' button to click on it
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# Esperar a que se complete la carga de la página principal o manejar ventanas emergentes
try:
    # Detección de pop-up de "Guardar información de inicio de sesión"
    save_info_popup = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Ahora no')]"))
    )
    save_info_popup.click()
    print("Pop-up de guardar información de login fue manejado.")
except Exception as e:
    print(f"No se pudo manejar el pop-up: {e}")

try:
    # Detección de pop-up de notificaciones (con los botones "Activar" y "Ahora no")
    notifications_popup = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Ahora no')]"))
    )
    notifications_popup.click()
    print("Pop-up de notificaciones fue manejado.")
except Exception as e:
    print(f"No se pudo manejar el pop-up: {e}")

# Access your profile
account = 'https://www.instagram.com/' + user_
driver.get(account) 
# A partir de aquí, deberías estar logueado y puedes continuar con tu scraping


Pop-up de guardar información de login fue manejado.
Pop-up de notificaciones fue manejado.


# 3.- Get number of followers and following

In [3]:
# Usando un XPath relativo para encontrar los seguidores y seguidos
followers_xpath = "//section/main//header//ul/li[2]//a/span"
following_xpath = "//section/main//header//ul/li[3]//a/span"

# Extraer el número de seguidores con espera
try:
    num_followers = int(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, followers_xpath))
    ).text.replace(',', ''))
    print('Número de seguidores:', int(num_followers))
except Exception as e:
    print(f"No se pudo obtener el número de seguidores: {e}")

# Extraer el número de seguidos con espera
try:
    num_following = int(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, following_xpath))
    ).text.replace(',', ''))
    print('Número de seguidos:', int(num_following))
except Exception as e:
    print(f"No se pudo obtener el número de seguidos: {e}")


Número de seguidores: 134
Número de seguidos: 373


In [4]:
# Hacer clic en el enlace de seguidores
try:
    driver.find_element(By.XPATH, followers_xpath).click()
    print("Opción 'seguidores' clicada.")
except Exception as e:
    print(f"No se pudo hacer clic en 'seguidores': {e}")


Opción 'seguidores' clicada.


# 4.- Extract follower accounts

In [6]:

# Detect pop-up window
pop_up_xpath = '//html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]'
pop_up_window = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pop_up_xpath)))

# Scroll through all following
b=0
while num_followers > b:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', pop_up_window)
    b=len(driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade'))
    #print(b)
    time.sleep(1)
    


# Listar mis seguidores:

In [7]:
# Get followers accounts
get_followers = driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade')

followers = []
for i in get_followers:
    seguidor = i.text
    followers.append(seguidor) 
print(f'Seguidores: {len(followers)}\n\n', followers)

Seguidores: 134

 ['susan_suarezhuatangari', 'katherinehuatangari', 'evelymxuxa', 'buinilina', 'escalanteesquivelarquitectos', 'gladibeth.28', 'jonathanbovi', 'elizabeth01397', 'elrincondekev', 'ilse_asesorinmobiliario', 'codeguardian', 'leninhurtado16', 'juanlimacordova', 'mganghela', 'nilso_eiinzi_h', 'dorissayagocordova', 'reflejosspa', 'edithicitarg', 'ttatianamera', 'leydibuenocamizan', 'antoniolimacordova', 'izela.tl', 'linconmacedo9', 'delita.cano', 'anyila.yamile', 'network_stream', 'edu.fernandez_30', 'alexbustamantesantacruz', 'nicodemos_salazar_23', 'mirella.zuloeta', 'laura_wendlingj', 'it_samanthaweb', 'nortmedic.uniformes', 'forex_trading_peru', 'devops_stagelatam', 'gig.a.dev', 'joel.siesquen', 'sonqorest', 'cespedesmarthaa', 'charlychavezgue', 'a1nz2802', 'mariaedisitacamisanperez', 'erpsol_mex', 'gonzalezannyely', 'marbelispa2024', '_kathee7', 'cumbialisa', 'adriangmori', 'mita.cordova', 'mjames4719', 'ghost_jea', 'edgard.prieto', 'rociocamizan', 'motosjch', 'yahairato

In [8]:
# Access your Instagram account
driver.get(account)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, following_xpath))).click()
print("Opción 'seguidos' clicada.")

#driver.find_element(By.XPATH, following_xpath).click()


Opción 'seguidos' clicada.


In [10]:
# Detect pop-up window
pop_up_xpath = '//html/body/div[6]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[4]'
pop_up_window = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, pop_up_xpath)))

# Scroll through all following
b=0
while num_following > b:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', pop_up_window)
    b=len(driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade'))
    #print(b)
    time.sleep(1)


In [11]:
# Get following accounts
get_following = driver.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacw._aacx._aad7._aade')

following = []
for i in get_following:
    seguido = i.text
    following.append(seguido) 
print(f'Following: {len(following)}\n\n', following)

Following: 373

 ['rbrtzin.usd', 'brenda_chero07', 'susan_suarezhuatangari', 'kevincelis7', 'steven_vera_c', 'cherilsaavedra', 'littleangee', 'emilyjhasminalvarado', 'anyila.yamile', 'mita.cordova', 'a1nz2802', 'leylidi_alarcon', 'rociocamizan', 'xiomara.piscoya', 'ttatianamera', 'gabyfashion1601', 'alonso_martinez09', 'andleo17', 'victor.0404', 'yarihana_hs', 'alexbustamantesantacruz', 'caritorold', 'javierrodasn', 'katherlu19', 'raftjaz', 'tayanahuatangare', 'ysamar_gonzales', 'edenilsonfloresar', 'leobar_37', 'izela.tl', 'adriangmori', 'mileralvarez0598', 'manuel_llontop2409', 'leninhurtado16', 'freddy_bn', 'edithicitarg', 'laurasanchezalvares', 'yesseniacubasq19', 'estherbazan_', 'oxalcvasquez', 'charlychavezgue', 'arumi.naomi', 'dorissayagocordova', 'chinskb', 'antopizaarro', 'wilitorrespasapera', 'tatiherreraracho', 'mendozagalangabriel', 'katherinehuatangari', 'naomy2307', 'reflejosspa', 'cesar.ribe', 'gladibeth.28', 'atuc_z', 'elizabeth01397', 'nilso_eiinzi_h', 'keyssi_fernande

# Step 5: Identify accounts you follow but don't follow you back

In [12]:
# Limpiar las listas eliminando espacios en blanco antes y después de los nombres
clean_followers = [follower.strip() for follower in followers]
clean_following = [follow.strip() for follow in following]

# Calcular quiénes no te siguen
non_followers = list(set(clean_following) - set(clean_followers))

# Mostrar la cantidad de usuarios que no te siguen y sus perfiles
print(f'Usuarios que no me SIGUEN: {len(non_followers)}\n')

# Formatear la salida con enlaces a sus perfiles
for user in non_followers:
    print(f'{user: <50} | https://instagram.com/{user}')


Usuarios que no me SIGUEN: 268

ey_peru                                            | https://instagram.com/ey_peru
arnold_robra12                                     | https://instagram.com/arnold_robra12
car.men4272                                        | https://instagram.com/car.men4272
meraaida                                           | https://instagram.com/meraaida
vanessa.flores.03                                  | https://instagram.com/vanessa.flores.03
condorcoders                                       | https://instagram.com/condorcoders
kristhieramos                                      | https://instagram.com/kristhieramos
abigailvg.18                                       | https://instagram.com/abigailvg.18
cristhianlimabueno11                               | https://instagram.com/cristhianlimabueno11
jadelacruzg                                        | https://instagram.com/jadelacruzg
mirtharojas95                                      | https://instagram.com/mirtharo